*Note that this notebook is SQL and may be executed using a Servless SQL Warehouse* 

In [0]:
DECLARE OR REPLACE VARIABLE catalog_use STRING DEFAULT 'redox';
DECLARE OR REPLACE VARIABLE schema_use STRING DEFAULT 'hls_webinar_fy25q4';

In [0]:
SET VAR catalog_use = :catalog_use;
SET VAR schema_use = :schema_use; 

In [0]:
USE IDENTIFIER(catalog_use || '.' || schema_use);
SELECT current_catalog(), current_schema();

In [0]:
CREATE OR REPLACE FUNCTION redox_request 
(
  http_method STRING COMMENT "The http method to invoke for the Redox Rest API. Typically 'post' or 'get'." 
  ,resource STRING COMMENT "The Redox resource endpoint to use for the REST API. (Note including the base_url.)" 
  ,action STRING COMMENT "The Redox action to invoke for the resource. (e.g. For ClaimResponse, the action is <dollar_sign>respond.)" 
  ,data STRING COMMENT "The data to send to the Redox API. This is typically a JSON string but may be a base64 encoded string." 
)
RETURNS STRING
LANGUAGE SQL
NOT DETERMINISTIC
CONTAINS SQL
COMMENT 'The response text from the Redox API Call.'
RETURN ai_query(
  endpoint => 'redox_make_request'
  ,request => named_struct(
    'http_method', http_method
    ,'resource', resource
    ,'action', action
    ,'data', data
  )
  ,returnType => "STRING"
);

In [0]:
with eg as (
  SELECT
    "post" as http_method
    ,"DiagnosticReport" as resource
    ,"_search" as action
    ,"subject=Patient/81c2f5eb-f99f-40c4-b504-59483e6148d7" as data
)
,response as (
  SELECT
    try_parse_json(redox_request(
    http_method
    ,resource
    ,action
    ,data
  )) as response
FROM 
  eg
)
SELECT
  entry.value:fullUrl::string as fullUrl
  ,entry.value:resource.based_on
  ,entry.value:resource.code
  ,entry.value:resource.encounter.reference::string as encounter_url
  ,entry.value:resource.id::string as diagnostic_report_id
  ,entry.value:resource.issued::timestamp as issued
  ,entry.value:resource.meta
  ,presentedForm.value:contentType::string as content_type
  ,presentedForm.value:creation::timestamp as creation
  ,unbase64(presentedForm.value:data::string)::string as diagnostic_report
FROM 
  response 
  ,LATERAL variant_explode(response:entry) as entry
  ,LATERAL variant_explode(entry.value:resource.presentedForm) as presentedForm
;